In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.animation import FuncAnimation
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from IPython.display import HTML, display
import textwrap
import warnings
import os

warnings.filterwarnings('ignore')
sns.set_theme(style="whitegrid")
plt.rcParams['figure.facecolor'] = 'white'

class MarketMasterAnalyzer:
    def __init__(self, filepath):
        self.df = None
        self.filepath = filepath
        self.load_data()

    def load_data(self):
        if not os.path.exists(self.filepath):
            print(f"\n[!] Error: File '{self.filepath}' not found.")
            return False
        try:
            self.df = pd.read_csv(self.filepath)
            self.df.columns = [c.replace('_x0020_', ' ') for c in self.df.columns]
            rename_map = {'Modal Price': 'Modal_Price', 'Min Price': 'Min_Price', 'Max Price': 'Max_Price'}
            self.df.rename(columns=rename_map, inplace=True)
            for col in ['Modal_Price', 'Min_Price', 'Max_Price']:
                if col in self.df.columns:
                    self.df[col] = pd.to_numeric(self.df[col], errors='coerce')
            self.df.dropna(subset=['Modal_Price', 'Commodity', 'Market'], inplace=True)
            print(f"\n[✓] Successfully loaded {len(self.df)} records from {self.filepath}")
            return True
        except Exception as e:
            print(f"\n[!] Load Error: {e}")
            return False

    def get_selection(self):
        items = sorted(self.df['Commodity'].unique().tolist())
        print("\n--- Available Commodities ---")
        for i, name in enumerate(items[:15], 1):
            print(f"{i}. {name}")
        val = input("\nEnter Name or Number: ").strip()
        if val.isdigit() and 0 < int(val) <= len(items):
            return items[int(val)-1]
        for name in items:
            if val.lower() in name.lower(): return name
        return "Brinjal"

    def option_1_distribution(self):
        item = self.get_selection()
        subset = self.df[self.df['Commodity'] == item]
        data = subset.groupby(['State', 'Market'])['Modal_Price'].mean().sort_values(ascending=False).head(10)
        y_labels = ["\n".join(textwrap.wrap(f"{s} ({m})", width=25)) for s, m in data.index]
        prices = data.values.tolist()
        fig, ax = plt.subplots(figsize=(12, 8))
        plt.subplots_adjust(left=0.35)
        bars = ax.barh(y_labels[::-1], [0]*len(y_labels), color=sns.color_palette("magma", len(y_labels)))
        ax.set_xlim(0, max(prices) * 1.2)
        ax.set_title(f"Market-wise Price Distribution: {item}", weight='bold', pad=20)
        def update(f):
            for i, (b, p) in enumerate(zip(bars, prices[::-1])):
                b.set_width(p * (f + 1) / 30)
                if f == 29: ax.text(p + 5, i, f"₹{int(p)}", va='center', weight='bold')
            return bars
        ani = FuncAnimation(fig, update, frames=30, interval=40, blit=False)
        display(HTML(ani.to_jshtml()))
        plt.close()

    def option_2_heatmap(self):
        t_c = self.df.groupby('Commodity')['Modal_Price'].mean().nlargest(8).index
        t_m = self.df.groupby('Market')['Modal_Price'].mean().nlargest(12).index
        piv = self.df[self.df['Commodity'].isin(t_c) & self.df['Market'].isin(t_m)].pivot_table(index='Market', columns='Commodity', values='Modal_Price')
        plt.figure(figsize=(12, 8))
        sns.heatmap(piv, annot=True, fmt='.0f', cmap='YlGnBu')
        plt.show()

    def option_3_outliers(self):
        item = self.get_selection()
        data = self.df[self.df['Commodity'] == item]['Modal_Price']
        plt.figure(figsize=(10, 6))
        sns.boxplot(x=data, color='salmon', fliersize=8)
        sns.stripplot(x=data, color='black', alpha=0.3, size=4)
        plt.title(f"Statistical Outlier Detection: {item}", weight='bold')
        plt.xlabel("Price (₹)")
        plt.show()

    def option_4_volatility(self):
        v = self.df.groupby('Commodity')['Modal_Price'].agg(['mean', 'std']).dropna()
        v['vol'] = (v['std'] / v['mean']) * 100
        top = v.nlargest(10, 'vol')
        plt.figure(figsize=(10, 6))
        sns.barplot(x=top['vol'], y=top.index, palette='Reds_r')
        plt.show()

    def option_5_clustering(self):
        f = self.df.groupby('Commodity')['Modal_Price'].agg(['mean', 'std']).fillna(0)
        s = StandardScaler().fit_transform(f)
        f['Cluster'] = KMeans(n_clusters=3).fit_predict(s)
        plt.figure(figsize=(10, 6))
        sns.scatterplot(data=f, x='mean', y='std', hue='Cluster', s=100, palette='Set1')
        plt.show()

    def option_6_stability(self):
        stats = self.df.groupby('Commodity')['Modal_Price'].agg(['mean', 'std', 'count'])
        stats = stats[stats['count'] > 2].dropna()
        stats['stability'] = 1 / (stats['std'] / stats['mean'])
        top = stats.nlargest(10, 'stability')
        plt.figure(figsize=(10, 6))
        sns.barplot(x=top['stability'], y=top.index, palette='viridis')
        plt.show()

def main():
    print("="*40)
    print("   AGRI-DATA FILE LOADER")
    print("="*40)
    user_file = input("Enter CSV filename (or press Enter for 'Datasetused.csv'): ").strip()
    
    if user_file == "":
        user_file = "Datasetused.csv"
        
    app = MarketMasterAnalyzer(user_file)
    
    # Check if data loaded successfully before starting loop
    if app.df is None:
        print("\nExiting. Please make sure the CSV file is in the same folder.")
        return

    menu = {
        '1': app.option_1_distribution, 
        '2': app.option_2_heatmap, 
        '3': app.option_3_outliers, 
        '4': app.option_4_volatility, 
        '5': app.option_5_clustering, 
        '6': app.option_6_stability
    }
    
    while True:
        print("\n" + "="*35)
        print("   AGRICULTURE MARKET ANALYZER")
        print("="*35)
        print("1. Market-wise Price Distribution")
        print("2. Heatmap")
        print("3. Outlier Detection")
        print("4. Volatility")
        print("5. Clustering")
        print("6. Stability")
        print("7. Exit")
        
        c = input("\nEnter choice (1-7): ").strip()
        
        if c == '7':
            print("\nExiting the system. Goodbye!")
            break
        elif c in menu:
            menu[c]()
        else:
            print("\n[!] Invalid choice. Please enter 1-7.")

if __name__ == "__main__":
    main()